In [13]:
!pip install tinyimagenet
!pip install torchvision
!pip install seaborn
!pip install torchmetrics
!pip install poutyne

from tinyimagenet import TinyImageNet
from pathlib import Path
import torch
from torchvision import models


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


/home/facundoq/.virtualenvs/tinyimagenet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:


weights = models.EfficientNet_V2_S_Weights.IMAGENET1K_V1
base_model = models.efficientnet_v2_s(weights=weights)
tinyimagenet_classes = 200
base_model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2,inplace=True),
    torch.nn.Linear(1280, tinyimagenet_classes),
)

model = torch.nn.Sequential(
    weights.transforms(),
    base_model,
    
)
print(base_model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [14]:
import poutyne


device = torch.device("cpu")
def get_dataset(split): return TinyImageNet(Path("~/.torchvision/tinyimagenet/"),split=split,imagenet_idx=False)

train_dataset = get_dataset("train")
val_dataset = get_dataset("val")


poutyne_model = poutyne.Model(
        model,
        'adam',
        'cross_entropy',
        batch_metrics=['accuracy',poutyne.TopKAccuracy(5)],
        epoch_metrics=['f1'],
        device=device
)


poutyne_model.fit_dataset(train_dataset,valid_dataset=val_dataset,batch_size=32,epochs=5)

/home/facundoq/.virtualenvs/tinyimagenet/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 1/1 Step:    17/50000   0.03% |                    |ETA: 17h49m28.44s loss: 5.468174 acc: 0.000000 

KeyboardInterrupt: 